In [586]:
ls_columns_result = [
    'Врем. сопротивление',
    'прогноз Врем. сопротивление',
    'MAE Врем. сопротивление',
#     'MSE Врем. сопротивление',
    'Предел текучести',
    'прогноз Предел текучести', 
    'MAE Предел текучести',
#     'MSE Предел текучести',
]

ls_columns_output = [
    'Дата термообработки',
    '№ партии',
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'B',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый', 
    'Параметр отпуск новый 2',
    'Параметр отпуск новый V',
    'Величина зерна',
    'Тип предела текучести (1186)',
    'ICD'
    ]

ls_columns_required = [
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'Тип предела текучести (1186)'
]

ls_opt_need = ['Гр. прочн.', 'марка стали', 'толщина стенки', 'диаметр']

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os

importing Jupyter notebook from C:\Users\kseniya.yuldasheva\Desktop\gitPNTZtoyuk_1\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


In [105]:
load = datetime.now()
database = pd.read_csv('prepared_to_saw_gp.csv', low_memory=False)
print('База исторических режимов загружена за ', datetime.now() - load)


replace_dict_gr = {
    ' ':'',
    '/':'-',
    'ТИП':'тип',
    'К':'K', #русский на английский, везде
    'С':'C',
    'Р':'P',
    'Х':'X',
    'Е':'E',
    'Т':'T',
    'М':'M'
}


def fix_h_group(st):
    st = str(st)
    st = st.upper()
    for it in replace_dict_gr:
        st = st.replace(it, replace_dict_gr[it])
    return st

                
def close_value(database, col, value):
    database['diff'] = np.abs(database[col]-value)
    return database.loc[(database['diff']).argmin(),:][col]

def find_row_close_sort(database, row):
    for col in ls_opt_need:
        tmp = database[database[col]==row[col]]
        if tmp.shape[0]>0:
            database = tmp.copy()
        else:
            try:
                value = close_value(database, col, row[col])
                database = database[database[col]==value]
            except TypeError:
                database = database[database[col].apply(lambda x: str(x).split('-')[0]) == row[col].split('-')[0]]
#                 database = database[database[col].apply(lambda x: str(x).split(' ')[0]) == row[col].split(' ')[0]]
                if database.shape[0]==0:
                    tmp = row
                    tmp[[col for col in row.index if col not in ls_opt_need]] = None
                    tmp['№ партии'] = 'Error!!! (причина:'+ col+')'
                    return tmp
                
    return pd.Series(database[database['Дата термообработки'] == database['Дата термообработки'].max()].iloc[0, :])


def find_close_sort(database, df):
    database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)   
    df['Гр. прочн.'] = df['Гр. прочн.'].apply(fix_h_group)
    df = df.apply(lambda x: find_row_close_sort(database, x), axis=1)
    return df

# возвращает модель, список признаков, по которому строилась модель
def load_model(dir_name):
    model = pickle.load(open(dir_name+'/RF_model_'+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
    return model, ls_need_col


def make_result_valid_file(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    result = pd.DataFrame()
    base = pd.DataFrame()
    model = []
    ls_need_col = []
    
    model, ls_need_col = load_model(dir_names)
    time = datetime.now()
    valid = clean_data(file_name, ls_columns_output)
    valid.reset_index(drop=True, inplace=True)

    print('Введенные данные очищены за ', datetime.now() - time)
    time = datetime.now()

    try:
        y_valid = valid[targets].copy()
    except KeyError:
        y_valid = [0 for x in range(0,valid.shape[0])]
    try:
        X_valid = valid[ls_columns_output].copy()
    except KeyError:
        X_valid = valid[ls_need_col].copy()

    X_valid_c = X_valid[ls_need_col].copy()
    time = datetime.now()

    y_pred = model.predict(X_valid_c)
    print('Предсказания ', y_pred.shape[0], ' строчек за ', datetime.now() - time)

    for i in range(len(targets)):
        result[targets[i]] = y_valid[targets[i]]
        result['прогноз '+ targets[i]] = y_pred[:, i]
        for x in range(result[targets[i]].shape[0]):
            if result.loc[x, targets[i]] != 0:
                result.loc[x, 'MAE '+ targets[i]] = np.abs(result.loc[x, targets[i]] - result.loc[x, 'прогноз '+ targets[i]])
            else:
                result.loc[x, 'MAE '+ targets[i]] = ''


    time = datetime.now()
    base = find_close_sort(database, X_valid)
    print('Найдены базовые исторические режимы за ', datetime.now() - time)
    time = datetime.now()
    result = pd.concat([result, X_valid], axis=1)
    time = datetime.now()
    base.index = [str(i) + ' исторический' for i in range(len(base.index))]
    con = pd.DataFrame()
    time = datetime.now()
    missing = ['-' for i in result.index]
    missing = pd.DataFrame(missing)
    for i, j, k in zip(result.index, base.index, missing.index):
        con = pd.concat([con, result[result.index == i], base[base.index == j], missing[missing.index == k]])
    con = con[ls_columns_result + ls_columns_output]
    con.to_excel(output_filename)
    return con, y_valid, y_pred

file_name =  'C:\\Users\kseniya.yuldasheva\Desktop\PNTZ_64\input\input.xlsx'
targets = ['Предел текучести', 'Врем. сопротивление']
dir_names = os.getcwd()+'/DATA/MODELS_RF/H+YS+BATH GS'

now = datetime.now()
time = "%d_%ddate %d_%d_%dtime" % (now.day, now.month, now.hour, now.minute, now.second)
output_filename = 'C:/Users/kseniya.yuldasheva/Desktop/PNTZ_64/output/' + "_" + time + ".xlsx"

time = datetime.now()
df, y_valid, y_pred = make_result_valid_file(file_name, dir_names, targets, output_filename)
print('Время работы скрипта на ', int(df.shape[0]/3), 'строчках: ', datetime.now() - time)

База исторических режимов загружена за  0:00:03.047096
Index(['№ плавки', '№ партии', 'Дата форм. цехом',
       'Тип предела текучести (1186)', 'Врем. сопротивление',
       'Предел текучести', 'НД (гост, ту)', 'марка стали', 'диаметр',
       'толщина стенки', 'Гр. прочн.', '1 зона по ВТР закалка',
       '2 зона по ВТР закалка', '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'время выдержки в закалочной ванне, сек.',
       't˚ C трубы после ванны', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'масса\nтн', 'шаг балок закалочная печь, сек',
       'шаг балок отпускная печь, сек', 'Поставщик', 'C', 'Mn', 'Si', 'P', 'S',
       'Cr', 'Ni', 'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'B', 'ICD',
       'C-coef', 'Параметр закалка', 'Параметр отпуск',
       'Параметр отпуск новый', 'Параметр от

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Найдены базовые исторические режимы за  0:00:01.092932


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api

Время работы скрипта на  13 строчках:  0:00:03.112273


In [121]:
d = clean_data('C:\\Users\kseniya.yuldasheva\Desktop\PNTZ_64\input\input.xlsx', ls_columns_output)

Index(['№ плавки', '№ партии', 'Дата форм. цехом',
       'Тип предела текучести (1186)', 'Врем. сопротивление',
       'Предел текучести', 'НД (гост, ту)', 'марка стали', 'диаметр',
       'толщина стенки', 'Гр. прочн.', '1 зона по ВТР закалка',
       '2 зона по ВТР закалка', '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'время выдержки в закалочной ванне, сек.',
       't˚ C трубы после ванны', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'масса\nтн', 'шаг балок закалочная печь, сек',
       'шаг балок отпускная печь, сек', 'Поставщик', 'C', 'Mn', 'Si', 'P', 'S',
       'Cr', 'Ni', 'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'B', 'ICD',
       'C-coef', 'Параметр закалка', 'Параметр отпуск',
       'Параметр отпуск новый', 'Параметр отпуск новый 2',
       'Параметр отпуск новый V', 'Отнош

In [611]:

ls_opt_need = ['марка стали', 'толщина стенки', 'диаметр']

replace_dict_gr = {
    ' ':'',
    '/':'-',
    'ТИП':'тип',
    'К':'K', #русский на английский, везде
    'С':'C',
    'Р':'P',
    'Х':'X',
    'Е':'E',
    'Т':'T',
    'М':'M'
}


def fix_h_group(st):
    st = str(st)
    st = st.upper()
    for it in replace_dict_gr:
        st = st.replace(it, replace_dict_gr[it])
    return st

                
def close_value(database, col, value):
#     print(database.shape)
#     print(value, type(value))
    database['diff'] = np.abs(database[col]-value)
    database = database[database['diff'] <= 3]
    if database['diff'].shape[0] > 0:
#         return database.loc[(database['diff']).argmin(),:][col]
        return set(database[col])
    else:
        return 

def find_row_close_sort(database, row):
    for col in ls_opt_need:
#         print(database.shape)
        try:
            value = close_value(database, col, row[col])
            con = pd.DataFrame()
#             print(value)
            for v in list(value):
#                 print(v)
                con = pd.concat([con, database[database[col] == v]])
#             database = database[database[col]==value]
            database = con
#             print(database.shape, row[col], col)
        except TypeError:
#             print('ТЕ', database.shape, row[col], col)
            database = database[database[col].apply(lambda x: str(x).split('-')[0]) == row[col].split('-')[0]]
#             database = database[database[col].apply(lambda x: str(x).split(' ')[0]) == row[col].split(' ')[0]]
#             print('ТЕ', database.shape, row[col], col)
#             if database.shape[0]==0:
#                 tmp = row
#                 tmp[[col for col in row.index if col not in ls_opt_need]] = None
#                 tmp['№ партии'] = 'Error!!! (причина:'+ col+')'
#                 return tmp
    if database.shape[0]>0:
        database = database[database['Дата термообработки'] == database['Дата термообработки'].max()].iloc[0, :]
    return database[ls_columns_output]


def find_last_close_sort(database, df):
    database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)   
    df['Гр. прочн.'] = df['Гр. прочн.'].apply(fix_h_group)
    tmp = pd.DataFrame()
    for row in df.index:
        if not find_row_close_sort(database, df.iloc[row, :]).empty:
#             print(tmp.shape, find_row_close_sort(database, df.iloc[row, :]).shape)
            tmp = pd.concat([tmp, pd.DataFrame(find_row_close_sort(database, df.iloc[row, :])).T])
    tmp = pd.DataFrame(tmp)
#     df = df.apply(lambda x: find_row_close_sort(database, x), axis=1)
    return tmp

In [618]:
b = find_last_close_sort(database, d)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


(0, 0) (202,)
(1, 202) (202,)
(2, 202) (202,)
(3, 202) (202,)
(4, 202) (202,)
(5, 202) (202,)
(6, 202) (202,)
(7, 202) (202,)
(8, 202) (202,)
(9, 202) (202,)
(10, 202) (202,)
(11, 202) (202,)
(12, 202) (202,)


In [305]:
d

,№ партии,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",...,Параметр закалка,Параметр отпуск,Параметр отпуск новый,Параметр отпуск новый 2,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),ICD,Предел текучести,Врем. сопротивление
0,nan,20А-пр,159.0,6.0,K50-1,865,865,870,24.000000,0.65,...,849.160213,18.731820,49.518914,13.387381,12.864400,10.138389,0.2,0.530196,0.000000,0.000000
1,6f7316,13ХФА,219.0,18.0,K50-1,935,935,945,45.000000,0.10,...,937.779104,19.247291,50.145301,13.513888,12.943627,17.168553,0.2,0.890914,42.500000,55.750000
2,7f0138,13ХФА,219.0,18.0,K50-2,935,935,945,40.000000,0.10,...,934.669104,18.754021,50.028278,13.591845,13.032934,15.114382,0.2,0.631831,42.250000,56.250000
3,v22400,30Г2,88.9,12.2,K50-3,900,900,910,40.000000,0.10,...,900.251927,16.514232,49.630275,10.687857,10.288549,21.804989,0.5,1.163209,64.081633,77.397959
4,1402,37Г2Ф,194.5,14.5,K50-4,880,880,890,40.000000,0.10,...,880.577434,18.899575,50.024196,11.291933,10.588942,10.053579,0.2,2.622215,67.500000,79.500000
5,1418,37Г2Ф,194.5,14.5,K50-5,880,880,890,40.000000,0.10,...,880.577434,19.094760,50.023977,11.156285,10.431171,9.911444,0.2,2.858220,65.000000,76.000000
6,5154,37Г2Ф,194.5,19.7,K50-6,880,880,890,40.000000,0.10,...,880.577434,18.289423,50.023478,11.009884,10.327215,9.270990,0.2,2.569336,79.500000,83.000000
7,5f1246,37Г2Ф,187.7,20.4,K50-7,880,880,890,43.333333,0.10,...,882.504093,18.754021,50.024193,10.917944,10.261723,16.455930,0.2,2.746515,70.500000,80.500000
8,5f0640,37Г2Ф,187.7,23.5,K50-8,880,880,890,45.000000,0.10,...,883.414751,19.025517,50.142782,11.354158,10.716862,16.499589,0.2,2.381384,67.000000,79.000000
9,5f0642,37Г2Ф,187.7,23.5,K50-9,880,880,890,45.000000,0.10,...,883.414751,18.958312,50.142416,11.292622,10.657576,14.435205,0.2,2.632277,69.000000,77.000000


In [615]:
c[ls_opt_need + ['Дата термообработки']]

,марка стали,толщина стенки,диаметр,Дата термообработки
0,20А-пр,6.0,114.0,2017-03-21
1,13ХФА,18.0,219.0,2017-01-20
2,13ХФА,18.0,219.0,2017-01-20
3,30Г2,12.2,88.9,2013-11-08
4,37Г2Ф,14.5,194.5,2018-03-28
5,37Г2Ф,14.5,194.5,2018-03-28
6,37Г2Ф,19.7,194.5,2017-10-10
7,37Г2Ф,20.4,187.7,2015-11-04
8,37Г2Ф,23.5,187.7,2016-10-15
9,37Г2Ф,23.5,187.7,2016-10-15


In [617]:
b[ls_opt_need + ['Дата термообработки']]

,марка стали,толщина стенки,диаметр,Дата термообработки
11604,20А,6,159,2017-05-20
19325,13ХФА,18,219,2017-01-20
19325,13ХФА,18,219,2017-01-20
17964,30Г2,12.2,88.9,2013-11-08
23529,37Г2Ф,14.5,194.5,2018-03-28
23529,37Г2Ф,14.5,194.5,2018-03-28
13706,37Г2Ф,19.7,194.5,2017-10-10
22693,37Г2Ф,19,187.7,2017-01-14
11105,37Г2Ф,21,187.71,2016-12-12
11105,37Г2Ф,21,187.71,2016-12-12


In [613]:
ls_opt_need = ['марка стали', 'толщина стенки', 'диаметр']

replace_dict_gr = {
    ' ':'',
    '/':'-',
    'ТИП':'тип',
    'К':'K', #русский на английский, везде
    'С':'C',
    'Р':'P',
    'Х':'X',
    'Е':'E',
    'Т':'T',
    'М':'M'
}


def fix_h_group(st):
    st = str(st)
    st = st.upper()
    for it in replace_dict_gr:
        st = st.replace(it, replace_dict_gr[it])
    return st

                
def close_value(database, col, value):
    database['diff'] = np.abs(database[col]-value)
    return database.loc[(database['diff']).argmin(),:][col]

def find_row_close_sort(database, row):
    for col in ls_opt_need:
        tmp = database[database[col]==row[col]]
        if tmp.shape[0]>0:
            database = tmp.copy()
        else:
            try:
                value = close_value(database, col, row[col])
                database = database[database[col]==value]
            except TypeError:
                database = database[database[col].apply(lambda x: str(x).split('-')[0]) == row[col].split('-')[0]]
                x = str(x).split('-')[0]
                database = database[database[col].apply(lambda x: str(x).split(' ')[0]) == row[col].split(' ')[0]]
                if database.shape[0]==0:
                    tmp = row
                    tmp[[col for col in row.index if col not in ls_opt_need]] = None
                    tmp['№ партии'] = 'Error!!! (причина:'+ col+')'
                    return tmp
                
    return pd.Series(database[database['Дата термообработки'] == database['Дата термообработки'].max()].iloc[0, :])


def find_close_sort(database, df):
    database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)   
    df['Гр. прочн.'] = df['Гр. прочн.'].apply(fix_h_group)
    df = df.apply(lambda x: find_row_close_sort(database, x), axis=1)
    return df


In [614]:
c = find_close_sort(database, d)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
